In [30]:
from z3 import *
import z3
import pandas as pd

import ortools.constraint_solver.pywrapcp as pywrapcp
from ortools.constraint_solver import routing_enums_pb2

import time

In [2]:
queries = []
with open("query_dump.smt2") as f:
    current = ""
    for line in f:
        if line == "(check-sat)":
            continue
        if line.strip() == "(reset)":
            queries.append(parse_smt2_string(current))
            current = ""
        current += line

len(queries)

61

In [60]:
def calc_distance_keep_order(first: list, second: list):
    i = 0
    while i < len(first) and i < len(second) and first[i].eq(second[i]):
        i += 1

    return len(first) - i + len(second) - i
    # return len(second) - i

distances = [[0] * len(queries)] * len(queries)
for i in range(len(queries)):
    for j in range(len(queries)):
        distances[i][j] = calc_distance_keep_order(queries[i], queries[j])

distances

In [62]:
manager = pywrapcp.RoutingIndexManager(len(distances), 1, 0)
routing = pywrapcp.RoutingModel(manager)

def distance_callback(from_index, to_index):
    """Returns the distance between the two nodes."""
    # Convert from routing variable Index to distance matrix NodeIndex.
    from_node = manager.IndexToNode(from_index)
    to_node = manager.IndexToNode(to_index)
    return distances[from_node][to_node]

transit_callback_index = routing.RegisterTransitCallback(distance_callback)
routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)
search_parameters = pywrapcp.DefaultRoutingSearchParameters()
search_parameters.first_solution_strategy = (
    routing_enums_pb2.FirstSolutionStrategy.AUTOMATIC)

def print_solution(manager, routing, solution):
    """Prints solution on console."""
    print('Objective: {} miles'.format(solution.ObjectiveValue()))
    index = routing.Start(0)
    plan_output = 'Route for vehicle 0:\n'
    route_distance = 0
    while not routing.IsEnd(index):
        plan_output += ' {} ->'.format(manager.IndexToNode(index))
        previous_index = index
        index = solution.Value(routing.NextVar(index))
        route_distance += routing.GetArcCostForVehicle(previous_index, index, 0)
    plan_output += ' {}\n'.format(manager.IndexToNode(index))
    print(plan_output)
    plan_output += 'Route distance: {}miles\n'.format(route_distance)

def get_vector(manager, routing, solution):
    index = routing.Start(0)
    route = []
    while not routing.IsEnd(index):
        route.append(manager.IndexToNode(index))
        index = solution.Value(routing.NextVar(index))

    return route

solution = routing.SolveWithParameters(search_parameters)
if solution:
    print_solution(manager, routing, solution)

best_case = get_vector(manager, routing, solution)

Objective: 629 miles
Route for vehicle 0:
 0 -> 60 -> 23 -> 2 -> 1 -> 5 -> 3 -> 46 -> 16 -> 12 -> 6 -> 4 -> 56 -> 31 -> 29 -> 28 -> 27 -> 26 -> 25 -> 22 -> 19 -> 15 -> 14 -> 13 -> 10 -> 7 -> 54 -> 52 -> 50 -> 49 -> 44 -> 43 -> 41 -> 38 -> 36 -> 35 -> 34 -> 32 -> 30 -> 20 -> 18 -> 17 -> 11 -> 8 -> 59 -> 58 -> 57 -> 55 -> 53 -> 51 -> 48 -> 47 -> 45 -> 42 -> 40 -> 39 -> 37 -> 33 -> 24 -> 21 -> 9 -> 0



In [71]:
# Base case

result_base = {}
solver = Solver()
total_time = 0.0
for i, q in enumerate(queries):
    start_time = time.perf_counter()
    result_base[i] = solver.check(q)
    end_time = time.perf_counter()
    total_time += end_time - start_time

print(solver.statistics())
print("Spent time:", total_time)

(:added-eqs           2783
 :arith-make-feasible 25
 :arith-max-columns   4
 :binary-propagations 170497
 :bv-bit2core         1627
 :bv-dynamic-eqs      3
 :bv->core-eq         2465
 :conflicts           3498
 :decisions           31210
 :del-clause          34464
 :final-checks        25
 :max-memory          20.55
 :memory              20.54
 :minimized-lits      4229
 :mk-bool-var         14090
 :mk-clause           38676
 :mk-clause-binary    1686
 :num-allocs          4250173383
 :num-checks          61
 :propagations        286369
 :restarts            24
 :rlimit-count        33481346)
Spent time: 0.06535585381789133


In [72]:
# Best case

solver = Solver()
last_query = []
result_best = {}

smt2 = ""

total_time = 0.0
total_pops = 0
total_pushes = 0

for qi in best_case:
    query = queries[qi]
    i = 0
    while i < len(last_query) and i < len(query) and last_query[i].eq(query[i]):
        i += 1
    print(i)

    solver.pop(len(last_query) - i)
    total_pops += len(last_query) - i

    for i in range(i, len(query)):
        solver.push()
        total_pushes += 1
        solver.add(query[i])
    
    start_time = time.perf_counter()
    result_best[qi] = solver.check()

    end_time = time.perf_counter()
    total_time += end_time - start_time

    last_query = query
    smt2 += solver.to_smt2()

assert(result_best == result_base)
print(solver.statistics())
print(total_time)
print(total_pops, total_pushes, sum(len(q) for q in queries))

0
0
0
0
1
0
1
0
3
0
0
3
0
0
1
0
0
0
0
0
4
0
0
0
0
0
3
0
0
0
0
0
0
3
0
0
0
0
0
0
0
0
0
1
2
0
0
0
0
0
0
1
1
0
0
0
0
0
0
2
0
(:added-eqs           4074
 :arith-make-feasible 25
 :arith-max-columns   4
 :bv-bit2core         4444
 :bv->core-eq         2115
 :conflicts           6449
 :decisions           67137
 :del-clause          103032
 :final-checks        25
 :max-memory          20.55
 :memory              20.38
 :minimized-lits      4561
 :mk-bool-var         37680
 :mk-clause           105088
 :num-allocs          4467674961.00
 :num-checks          61
 :propagations        491824
 :restarts            45
 :rlimit-count        34706559)
0.08010426221881062
255 261 287
